# Data viz ideas

## There's two main parts to this:
1) Dashboard building
2) Actual plots and such 
    - Most worn items per category 
    - Top colors worn
    - Parse data by season (winter/spring/summer/fall)

In [2]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import datetime
import calmap
import sheworewhat as sww

alt.data_transformers.disable_max_rows()
# alt.renderers.enable('notebook')

DataTransformerRegistry.enable('default')

In [3]:
closet = sww.closet_df()

closet

,ID,Item,Category,Sub-Category,Color,Pattern,Brand,Bought,Cost,2023,Price,PrimaryC,Name
0,0,Turtleneck,Top,Sweater,Black,Plain,Zara,"Secondhand, Thrifted",cheap,No,NaN,Black,0 Zara Turtleneck - Black
1,1,Tropical Tank,Top,Tanktop,"Black, Red, Gold",Feather,Plisse,"Secondhand, Thrifted",cheap,No,NaN,Black,1 Plisse Tropical Tank - Black
2,2,Leopard Tank,Top,Tanktop,"Black, Tan",Leopard,Plisse,"Secondhand, Thrifted",cheap,No,15.0,Black,2 Plisse Leopard Tank - Black
3,3,Jeans,Bottom,Pants,Blue,Plain,Aerie,New,cheap,No,50.0,Blue,3 Aerie Jeans - Blue
4,4,Shirt,Top,Shirt,"Black, White",Cheetah,Free People,"Secondhand, Depop",cheap,No,NaN,Black,4 Free People Shirt - Black
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,Baseball Cap,Accessory,Hat,White,Logo,Taylor Swift,New,pricy,No,35.0,White,92 Taylor Swift Baseball Cap - White
93,93,Baseball Cap,Accessory,Hat,White,Logo,Davis Farmer'S Market,New,cheap,No,NaN,White,93 Davis Farmer'S Market Baseball Cap - White
94,94,Sunglasses,Accessory,Glasses,"Black, Gold",Plain,Ray-Ban,New,expensive,No,160.0,Black,94 Ray-Ban Sunglasses - Black
95,95,Tights,Accessory,Tight,Black,Plain,H&M,New,cheap,No,NaN,Black,95 H&M Tights - Black


In [ ]:
acc_df, bottom_df, fb_df, out_df, shoes_df, top_df = sww.closet_cat(closet)

acc_df.tail(5)

## Closet EDA
- What percentage of my closet is new vs. secondhand?

In [ ]:
closet_count = closet.groupby(by="Bought").count()
closet_count

In [ ]:
closet_n = closet.groupby(["Bought"])[["Bought"]].agg('count').rename(columns={"Bought" : "N"}).reset_index()
closet_n["Percent"] = (closet_n['N'] / len(closet))
closet_n

Should I do just secondhand? then breakdown the secondhand

In [ ]:
closet_n = closet.groupby(["Bought"])[["Bought"]].agg('count').rename(columns={"Bought" : "N"}).reset_index()
closet_n["Percent"] = (closet_n['N'] / len(closet))

closet_n["Status"] = closet_n["Bought"].str.split(",").str[1]
closet_n = closet_n.replace(np.nan, "New")

closet_comp = alt.Chart(closet_n, title="Closet Composition"
                       ).mark_bar(color="Maroon"
                         ).encode(alt.X("Bought", axis=alt.Axis(labelAngle=-45), sort='-y'),
                                  alt.Y("Percent", axis=alt.Axis(format='%'),),
                                  alt.Tooltip("Percent", format=",.2f"),
                                  color=alt.condition(
                                    alt.datum.year == "New",
                                    alt.value('orange'),
                                    alt.value('maroon')   
                                    )
                                 )
                
closet_comp

In [ ]:
closet_n = closet.groupby(["Bought"])[["Bought"]].agg('count').rename(columns={"Bought" : "N"}).reset_index()
closet_n["Percent"] = (closet_n['N'] / len(closet))

closet_n["Purchased"] = closet_n["Bought"].str.split(",").str[0]
closet_n["Status"] = closet_n["Bought"].str.split(",").str[1]
closet_n = closet_n.replace(np.nan, "New")

closet_n

In [ ]:
closet_n.groupby(["Purchased"]).count()

In [ ]:
# new vs secondhand only plot
closet_comp = alt.Chart(closet_n, title="Closet Composition"
                       ).mark_bar(color="Maroon"
                         ).encode(alt.X("Bought", axis=alt.Axis(labelAngle=-45), sort='-y'),
                                  alt.Y("Percent", axis=alt.Axis(format='%'),),
                                  alt.Tooltip("Percent", format=",.2f"),
                                  color=alt.condition(
                                    alt.datum.year == "New",
                                    alt.value('orange'),
                                    alt.value('maroon')   
                                    )
                                 )
                
closet_comp

## clean and merge data
- need function to clean, merge, do counts and stuff for collected data

In [ ]:
path = "../data/2023TestData.csv"

df = pd.read_csv(path)
df

We don't need the timestamp, its redundant and easier to just use date

In [ ]:
df.iloc[:, 1:]

In [ ]:
df.drop("Timestamp", axis=1)

two things: would be useful to have counts and also the dates in a list per item

Need to be able to handle multiple items in a single category (layering, etc)
- doesnt make sense to edit google sheet because layering can happen with all categories

In [ ]:
path = "../data/2023TestData.csv"

df = pd.read_csv(path).drop("Timestamp", axis=1).melt("Date").dropna()

df

In [ ]:
df.groupby("value").count()

In [ ]:
df["test0"] = df["value"].str.split(",").str[0]
df["test1"] = df["value"].str.split(",").str[1]
df.head(3)

ugh maybe this can be reformatted somehow for easier parsing; also i dont love how the format is now

OR 
maybe if column contains more than 1 number, split it i

IDEA:
- GOOGLE SHEET FORM: ID ITEM - COLOR
- then multiple selections can be parse via "," in the sheet

In [ ]:
# def closet
path = "../data/2023TestData.csv"

df = pd.read_csv(path).drop("Timestamp", axis=1).melt("Date").dropna()

# extract ID number from value
df["ID"] = df.value.str.extract('(\d+)').astype(int)

closet_counts = df.groupby(["value", "ID"]).count().reset_index().rename(columns={"Date":"count"}).drop(["variable"],axis=1)

closet_counts

# left join closet + df
x = pd.merge(closet, closet_counts, how="left", on="ID")
x

In [ ]:
x = pd.merge(closet, df, on="ID")


In [ ]:
closet_counts = df.groupby(["value"]).count().reset_index().rename(columns={"Date":"count"}).drop(["ID", "variable"],axis=1)
closet_counts

In [ ]:
closet_comp = alt.Chart(closet_counts, title="2023 Most Worn Pieces"
                       ).mark_bar(color="Maroon"
                         ).encode(alt.X("value", axis=alt.Axis(labelAngle=-45), sort='-y'),
                                  alt.Y("count",),
                                  alt.Tooltip("value", format=",.2f"),
                                  color=alt.condition(
                                    alt.datum.year == "New",
                                    alt.value('orange'),
                                    alt.value('maroon')   
                                    )
                                 )
                
closet_comp

## Basic Wrangling Function
1. takes in google form data
2. counts how many of each item logged
3. left joins with closet data

In [ ]:
def complete_df(closet, path="../data/2023TestData.csv"):
    """
    Function to merge raw closet data and collected 2023 data.
    
    Parameters
    ----------
        closet : pandas.DataFrame
            Dataframe containing complete closet log.
        path : string
            String containing path of CSV of collected data.
    
    Returns
    -------
        complete_df : pandas.DataFrame
            Dataframe containing "ID", "Name", "count", "Item", 
            "Category", "Sub-Category", "Color", "Pattern", "Brand", "Cost", "2023"
    """


    form = pd.read_csv(path).drop("Timestamp", axis=1).melt("Date").dropna()

    # extract ID number from value
    form["ID"] = form.value.str.extract('(\d+)').astype(int)

    form_counts = form.groupby(["value", "ID"]).count().reset_index().rename(columns={"Date":"count"}).drop(["variable"],axis=1)
    
    # left join closet + df
    complete_df = pd.merge(closet, form_counts, how="left", on="ID")
    complete_df["Name"] = complete_df["Brand"] + " " + complete_df["Item"]
    complete_df = complete_df[["ID", "Name", "count", "Item", "Category", "Sub-Category", "Color", "Pattern", "Brand", "Cost", "2023"]]
    complete_df = complete_df.fillna(0).rename(columns={"count" : "Count"})
    complete_df["Count"] = complete_df["Count"].astype(int)
    
    return complete_df


In [ ]:
closet = sww.closet_df()
worn_df = complete_df(closet)
worn_df

In [ ]:
worn_df["Count"].astype(int)

## Rewriting function bc I'm a dumbass there is a much easier way to do this using value_counts()

In [41]:
path="../data/2023TestData.csv"

form = pd.read_csv(path).drop("Timestamp", axis=1).melt("Date").dropna()

# extract ID number from value
form["ID"] = form.value.str.extract('(\d+)').astype(int)
form.value_counts(["value"]).reset_index()

counts = form.value_counts(["value"]).reset_index().rename(columns={0 : "Count"})
pd.merge(form, counts,how="right", on="value")

,Date,variable,value,ID,Count
0,12/19/2022,Outerwear,84 Hollister Puffer - Green,84,10
1,12/20/2022,Outerwear,84 Hollister Puffer - Green,84,10
2,12/21/2022,Outerwear,84 Hollister Puffer - Green,84,10
3,12/22/2022,Outerwear,84 Hollister Puffer - Green,84,10
4,12/23/2022,Outerwear,84 Hollister Puffer - Green,84,10
...,...,...,...,...,...
99,12/23/2022,Bottom,4 Aerie Jeans - Blue,4,1
100,12/25/2022,Top,37 Sml Christmas Vest - Black,37,1
101,12/25/2022,Top2,33 No Brand Christmas Sweater - Black,33,1
102,12/26/2022,Top2,32 Rei Flannel - Orang,32,1


In [37]:
def complete_df(closet, path="../data/2023TestData.csv"):
    """
    Function to merge raw closet data and collected 2023 data.
    
    Parameters
    ----------
        closet : pandas.DataFrame
            Dataframe containing complete closet log.
        path : string
            String containing path of CSV of collected data.
    
    Returns
    -------
        complete_df : pandas.DataFrame
            Dataframe containing "ID", "Name", "count", "Item", 
            "Category", "Sub-Category", "Color", "Pattern", "Brand", "Cost", "2023"
    """


    form = pd.read_csv(path).drop("Timestamp", axis=1).melt("Date").dropna()

    # extract ID number from value
    form["ID"] = form.value.str.extract('(\d+)').astype(int)

    form_counts = form.groupby(["value", "ID"]).count().reset_index().rename(columns={"Date":"count"}).drop(["variable"],axis=1)
    
    # left join closet + df
    complete_df = pd.merge(closet, form_counts, how="left", on="ID")
    complete_df["Name"] = complete_df["Brand"] + " " + complete_df["Item"]
    complete_df = complete_df[["ID", "Name", "count", "Item", "Category", "Sub-Category", "Color", "Pattern", "Brand", "Cost", "2023"]]
    complete_df = complete_df.fillna(0).rename(columns={"count" : "Count"})
    complete_df["Count"] = complete_df["Count"].astype(int)
    
    return complete_df

## 2023 Most Worn Pieces

In [ ]:
most_worn = worn_df.nlargest(15, columns="Count")
closet_comp = alt.Chart(most_worn, title="2023 Most Worn Pieces"
                       ).mark_bar(color="Maroon"
                         ).encode(alt.X("Name", axis=alt.Axis(labelAngle=-45), sort="-y"),
                                  alt.Y("Count", 
                                        title="# of Times Worn",
                                        axis=alt.Axis(tickMinStep=1)),
                                  alt.Tooltip("Count")
                                 )
                
closet_comp

In [ ]:
def plot_mostworn():
    most_worn = worn_df.nlargest(15, columns="Count")
    closet_comp = alt.Chart(most_worn, title="2023 Most Worn Pieces"
                       ).mark_bar(color="Maroon"
                         ).encode(alt.X("Name",
                                        title="",
                                        axis=alt.Axis(labelAngle=-45), 
                                        sort="-y"),
                                  alt.Y("Count", 
                                        title="Times Worn",
                                        axis=alt.Axis(tickMinStep=1)),
                                  alt.Tooltip("Count")
                                 )
                
    return closet_comp

In [ ]:
plot_mostworn()

### Easier way to plot count 
- can use "count()" and make complete_df function less complex
- this doesn't seem to work unless you limit the x-axis data first so move on

In [ ]:
closet = sww.closet_df()

In [ ]:
form = pd.read_csv("../data/2023TestData.csv").drop("Timestamp", axis=1).melt("Date").dropna()

# extract ID number from value
form["ID"] = form.value.str.extract('(\d+)').astype(int)

# left join closet + df
complete_df = pd.merge(closet, form, how="left", on="ID")
complete_df["Name"] = complete_df["Brand"] + " " + complete_df["Item"]
complete_df = complete_df[["ID", "Name", "Item", "Category", "Sub-Category", "Color", "Pattern", "Brand", "Cost", "2023"]]


In [ ]:
closet_comp = alt.Chart(complete_df, title="2023 Most Worn Pieces"
                       ).mark_bar(color="Maroon"
                         ).encode(alt.X("Name", axis=alt.Axis(labelAngle=-45), sort="-y"),
                                  alt.Y("count()", 
                                        title="# of Times Worn",
                                        axis=alt.Axis(tickMinStep=1)),
                                  alt.Tooltip("count()")
                                 )
                
closet_comp

## Make similar plots but facet per category!

The problem with the below code is the x-axis is the same and we do not want that. We want the x axis to contain unique items for that category!
- Solution: generate 6 separate plots and concat

In [ ]:
category_worn = worn_df.nlargest(15, columns="Count")
closet_comp = alt.Chart(category_worn, title="2023 Most Worn Pieces"
                       ).mark_bar(color="Maroon"
                       ).encode(alt.X("Name", axis=alt.Axis(labelAngle=-45), sort="-y"),
                                alt.Y("Count", 
                                title="# of Times Worn",
                                axis=alt.Axis(tickMinStep=1)),
                                alt.Tooltip("Count")
                        ).facet("Category", columns=3)
                
closet_comp

In [ ]:
worn_df["Category"]

**Solution**: Concat six graphs because the x and y axes will inherently very different

*whyyyyy won't sort="y" work pls*

In [ ]:
categories = ["Top", "Bottom", "Full Body", "Outerwear", "Accessory", "Shoes"]

cat_plots = []

for i in categories:
    category_worn = worn_df.loc[worn_df["Category"] == i].nlargest(15, columns="Count")
    
    category_plot = alt.Chart(category_worn, title=f"2023 Most Worn {i}"
                       ).mark_bar(color="#B79492"
                       ).encode(alt.X("Name", title="", axis=alt.Axis(labelAngle=-45), sort="-y"),
                                alt.Y("Count", 
                                title="# of Times Worn",
                                axis=alt.Axis(tickMinStep=1)),
                                alt.Tooltip(["Name", "Count"])
                        ).resolve_scale(x='independent')
    cat_plots.append(category_plot)

# configure altair charts
row1 = alt.hconcat(cat_plots[0], cat_plots[1], cat_plots[2])
row2 = alt.hconcat(cat_plots[3], cat_plots[4], cat_plots[5])

category_plot = alt.vconcat(row1, row2)
category_plot

Troubleshooting this gd error

In [ ]:
category_worn = worn_df.loc[worn_df["Category"] == "Top"].nlargest(10, columns="Count")
category_worn["Count"]

In [ ]:
category_worn.head(5)

Weirdly the solution seems to be reduce the dataframe I'm pulling from?
`.nlargest(15, columns="Count")` to `nlargest.(5, columns="Count")` fixed it in the concat'd plot

In [ ]:
alt.Chart(category_worn, title=f"2023 Most Worn Top"
                   ).mark_bar(color="Pink"
                   ).encode(alt.X("Name", title="", axis=alt.Axis(labelAngle=-45), sort="-y"),
                            alt.Y("Count", 
                            title="# of Times Worn",
                            axis=alt.Axis(tickMinStep=1)),
                            alt.Tooltip(["Name", "Count"])
                    ).resolve_scale(x='independent')

In [ ]:
categories = ["Top", "Bottom", "Full Body", "Outerwear", "Accessory", "Shoes"]

cat_plots = []

for i in categories:
    category_worn = worn_df.loc[worn_df["Category"] == i].nlargest(5, columns="Count")
    
    category_plot = alt.Chart(category_worn, title=f"2023 Most Worn {i}"
                       ).mark_bar(color="#B40490"
                       ).encode(alt.X("Name", title="", axis=alt.Axis(labelAngle=-45), sort="-y"),
                                alt.Y("Count", 
                                title="# of Times Worn",
                                axis=alt.Axis(tickMinStep=1)),
                                alt.Tooltip(["Name", "Count"])
                        )
    cat_plots.append(category_plot)

# configure altair charts
row1 = alt.hconcat(cat_plots[0], cat_plots[1], cat_plots[2])
row2 = alt.hconcat(cat_plots[3], cat_plots[4], cat_plots[5])

category_plot = alt.vconcat(row1, row2)
category_plot

Make a plot function!

In [ ]:
def plot_facet():
    
    categories = ["Top", "Bottom", "Full Body", "Outerwear", "Accessory", "Shoes"]

    cat_plots = []

    for i in categories:
        category_worn = worn_df.loc[worn_df["Category"] == i].nlargest(5, columns="Count")

        category_plot = alt.Chart(category_worn, title=f"2023 Most Worn {i}"
                           ).mark_bar(color="#B40490"
                           ).encode(alt.X("Name", title="", axis=alt.Axis(labelAngle=-45), sort="-y"),
                                    alt.Y("Count", 
                                    title="# of Times Worn",
                                    axis=alt.Axis(tickMinStep=1)),
                                    alt.Tooltip(["Name", "Count"])
                            ).properties(
                                height=200,
                                width=150
                                )
        cat_plots.append(category_plot)

    # configure altair charts
    row1 = alt.hconcat(cat_plots[0], cat_plots[1], cat_plots[2])
    row2 = alt.hconcat(cat_plots[3], cat_plots[4], cat_plots[5])

    category_plot = alt.vconcat(row1, row2)
    return category_plot

In [ ]:
plot_facet()

## Heatmap of plots

In [ ]:
path = "../data/2023TestData.csv"

df = pd.read_csv(path).drop("Timestamp", axis=1).melt("Date").dropna()
df["Date"] = pd.to_datetime(df["Date"])

# extract ID number from value
df["ID"] = df.value.str.extract('(\d+)').astype(int)
df = pd.merge(closet, df, how="right", on="ID")
df = df[["ID", "Item", "Color", "Pattern", "Category", "Date"]]
df.sample(4)

Maybe once we have the top 10 items we can heatmap them?

In [ ]:
top_10 = [10, 85, 81]
heatmap_data = df.loc[df["ID"] == top_10[1]]
heatmap_data

Ok idea is:
- make a calender df
- full merge the above df
    - replace NA item with ..?
    - or make a boolean column that will be colored if yes? (0 1) 
- plot month on x and day on y axis 

In [ ]:
time_df = pd.DataFrame()
time_df["Date"] = pd.date_range('2022-12-01', periods=365)
time_df["Day"] = time_df["Date"].dt.day_name()


In [ ]:
year = pd.merge(time_df, heatmap_data, how="outer", on="Date")
year["Item"] = year["Item"].replace(np.nan, 0) 
year["Bool"] = np.where(year["Item"] == 0, 0, 1)

In [ ]:
year[15:22]

This is the ugliest plot I have seen in my life

In [ ]:
weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]
alt.Chart(year).mark_rect().encode(
    alt.X("Date:O", axis = alt.Axis(labelAngle=-45)), 
    alt.Y("Day", sort=weekdays), 
    color="Bool"
)

I wonder if it would be a better viz to see what months it was worn the most intead?
- bc yeah otherwise big x axis 

package: calmap

In [ ]:
year = year.set_index("Date")

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(12, 12))
cmap = calmap.yearplot(year["Bool"], cmap='BuPu', fillcolor='lightgray')
cmap

In [ ]:
week = time_df["Date"].dt.isocalendar()
time_df["Week"] = week["week"]

In [ ]:
weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]

time_df = pd.DataFrame()
time_df["Date"] = pd.date_range('2022-12-01', periods=365)
time_df["Day"] = time_df["Date"].dt.day_name()

year = pd.merge(time_df, heatmap_data, how="outer", on="Date")
year["Item"] = year["Item"].replace(np.nan, 0) 
year["Bool"] = np.where(year["Item"] == 0, 0, 1)

week = time_df["Date"].dt.isocalendar()
year["Week"] = week["week"].fillna(52)
year["Week"] = year["Week"].fillna(52)

year.tail(5)

In [ ]:
alt.Chart(year, title="Heatmap of Green Totebag Wears").mark_rect(
    stroke="white",
    strokeWidth=3,
    opacity=0.9
    ).encode(
    alt.X("Week:O", 
          axis = alt.Axis(labelAngle=-45)
         ), 
    alt.Y("Day", 
          sort=weekdays),
    alt.Color("Bool", 
              scale=alt.Scale(scheme='greenblue'),
              legend=None
             ),
    alt.Tooltip(["Date", "Day"])
    ).properties(
    height=200,
    width=700
    )

In [ ]:
# scale=alt.Scale(
#             domain=[-10,-5,0,5,9], 
#             range=['red','orange','white','lightblue','darkblue'], 
#             interpolate=method
#             )

In [ ]:
alt.Chart(year, title="Heatmap of Green Totebag Wears").mark_rect(
    stroke="white",
    strokeWidth=3,
    opacity=0.9,
    ).encode(
    alt.X("Week:O", 
          axis = alt.Axis(labelAngle=-45)
         ), 
    alt.Y("Day", 
          sort=weekdays),
    alt.Color("Bool", 
            scale=alt.Scale(
                domain=[0,1], 
                range=["#e0ddd5", "#7c9e7b"]),
                legend=None
             ),
    alt.Tooltip(["Date", "Day"])
    ).properties(
    height=200,
    width=600
    )

In [ ]:
year.iloc[16:18]

idea: fix week x axis to dates
- also what happened dec 18th lol

oh I think if the x axis is by week then everything will look better

In [ ]:
year['First_day'] = year['Date'] - year['Date'].dt.weekday * np.timedelta64(1, 'D')

# year["Week2"] = year["Week"].fillna(52)

year

In [ ]:
alt.Chart(year, title="Heatmap of Green Totebag Wears").mark_rect(
    stroke="white",
    strokeWidth=3,
    opacity=0.9,
    ).encode(
    alt.X("First_day:O", 
          axis = alt.Axis(labelAngle=-45,                         )
         ), 
    alt.Y("Day", 
          sort=weekdays),
    alt.Color("Bool", 
            scale=alt.Scale(
                domain=[0,1], 
                range=["#e0ddd5", "#7c9e7b"]),
                legend=None
             ),
    alt.Tooltip(["Date", "Day"])
    ).properties(
    height=200,
    width=600
    )

In [ ]:
week = time_df["Date"].dt.strftime('%m-%d-%y')

In [ ]:
week = time_df["Date"].dt.strftime('%m-%d-%y')
year['D'] = time_df["Date"].dt.strftime('%m-%d-%y')


In [ ]:
year["Week"] = year["First_day"].dt.strftime('%m-%d-%y')
year

In [ ]:
weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]

alt.Chart(year, title="Heatmap of Green Totebag Wears").mark_rect(
    stroke="white",
    strokeWidth=3,
    opacity=0.9,
    ).encode(
    alt.X("Week:O", 
          axis = alt.Axis(labelAngle=-45,                         )
         ), 
    alt.Y("Day", 
          sort=weekdays),
    alt.Color("Bool", 
            scale=alt.Scale(
                domain=[0,1], 
                range=["#e0ddd5", "#7c9e7b"]),
                legend=None
             ),
    alt.Tooltip(["Date", "Day"])
    ).properties(
    height=200,
    width=600
    )

## Edit Top 10 function
- to include dictionary of item id : item name

In [112]:
def test_top10(path="../data/2023Data.csv"):

    df = pd.read_csv(path).drop("Timestamp", axis=1).melt("Date").dropna()
    df["Date"] = pd.to_datetime(df["Date"])
    df["ID"] = df.value.str.extract("(\d+)").astype(int)

    # column of day of week for one calender year
    time_df = pd.DataFrame()
    time_df["Date"] = pd.date_range(df["Date"].min(), periods=365)
    time_df["Day"] = time_df["Date"].dt.day_name()

    # data wrangling to select top 10 most worn items
    closet = sww.closet_df()
    worn_df = sww.worn(closet)
    most_worn = worn_df.nlargest(10, columns="Count")

    # merge dataframes
    df = pd.merge(closet, df, how="right", on="ID")
    top_id = most_worn["ID"].to_list()
    top_item = (most_worn["Brand"] + " " + most_worn["Item"]).to_list()

    top_10 = dict(zip(top_id, top_item))
    return top_10, top_id, top_item, df # not sure which one i need yet lol

## Heatmap testing out and fixing for function

In [ ]:
# read in Google Form data
path = "../data/2023TestData.csv"
df = pd.read_csv(path).drop("Timestamp", axis=1).melt("Date").dropna()
df["Date"] = pd.to_datetime(df["Date"])
df["ID"] = df.value.str.extract('(\d+)').astype(int)

# column of day of week for one calender year 
time_df = pd.DataFrame()
time_df["Date"] = pd.date_range(df["Date"].min(), periods=365) 
time_df["Day"] = time_df["Date"].dt.day_name()

# merge dataframes
df = pd.merge(closet, df, how="right", on="ID")
df = df[["ID", "Item", "Color", "Pattern", "Category", "Date"]]

# select item 
top_10 = [10, 85, 81] # need to make this dynamic!! 
heatmap_data = df.loc[df["ID"] == top_10[1]]

year = pd.merge(time_df, heatmap_data, how="outer", on="Date")
year["Item"] = year["Item"].replace(np.nan, 0) 
year["Bool"] = np.where(year["Item"] == 0, 0, 1)

weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]

week = time_df["Date"].dt.isocalendar()
year["Week"] = week["week"].fillna(52)
year["Week"] = year["Week"].fillna(52)
year['First_day'] = year['Date'] - year['Date'].dt.weekday * np.timedelta64(1, 'D')

week = time_df["Date"].dt.strftime('%m-%d-%y')
year["Week"] = year["First_day"].dt.strftime('%m-%d-%y')


year.tail(5)

Ok not totally dynamic but the actual selection will change in plot

In [ ]:
closet = sww.closet_df()
worn_df = sww.complete_df(closet)
most_worn = worn_df.nlargest(10, columns="Count")

top_10 = most_worn["Count"].to_list() 
heatmap_data = df.loc[df["ID"] == top_10[1]] # need to make this dynamic in plot

In [ ]:
# read in Google Form data
def top_10_df(path = "../data/2023TestData.csv"):
    """
    Function to return IDs and counts of top 10 most worn items.
    
    Parameters:
    -----------
         path : str
            Path to CSV file containing closet information.
    
    Returns:
    --------
        top_10 : list
            List containing the IDs of the top 10 most worn items.
        
        df : pandas.DataFrame
            Dataframe containing data only for top 10 most worn items.
    """
    df = pd.read_csv(path).drop("Timestamp", axis=1).melt("Date").dropna()
    df["Date"] = pd.to_datetime(df["Date"])
    df["ID"] = df.value.str.extract('(\d+)').astype(int)

    # data wrangling to select top 10 most worn items
    closet = sww.closet_df()
    worn_df = sww.complete_df(closet)
    most_worn = worn_df.nlargest(10, columns="Count")
    
    # merge dataframes
    df = pd.merge(closet, df, how="right", on="ID")
    df = df[["ID", "Item", "Color", "Pattern", "Category", "Date", "Brand"]]

    top_10 = most_worn["ID"].to_list() 
 
    return top_10, df


In [96]:
i=0
# column of day of week for one calender year
time_df = pd.DataFrame()
time_df["Date"] = pd.date_range(df["Date"].min(), periods=365)
time_df["Day"] = time_df["Date"].dt.day_name()

heatmap_data = df.loc[df["ID"] == top_10[i]]  # need to make this dynamic in plot
item_name = heatmap_data["Brand"].iloc[0] + " " + heatmap_data["Item"].iloc[0]

# isolate item data
year = pd.merge(time_df, heatmap_data, how="outer", on="Date")
year["Item"] = year["Item"].replace(np.nan, 0)
year["Bool"] = np.where(year["Item"] == 0, 0, 1)

# this is horrible to read lol
# wrangling for prettier plotting
week = time_df["Date"].dt.isocalendar()
year["Week"] = week["week"].fillna(52)
year["Week"] = year["Week"].fillna(52)
year["First_day"] = year["Date"].dt.to_period("W-SAT").dt.start_time
week = time_df["Date"].dt.strftime("%m-%d-%y")
year["Week"] = year["First_day"].dt.strftime("%m-%d")
year = year[["Date", "Day", "Item", "ID", "Bool", "Week"]]

weekdays = [
    "Sunday",
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Sunday",
]
year.iloc[:15]


,Date,Day,Item,ID,Bool,Week
0,2023-01-01,Sunday,0,NaN,0,01-01
1,2023-01-02,Monday,Tennis Shoe,75.0,1,01-01
2,2023-01-03,Tuesday,Tennis Shoe,75.0,1,01-01
3,2023-01-04,Wednesday,0,NaN,0,01-01
4,2023-01-05,Thursday,0,NaN,0,01-01
5,2023-01-06,Friday,0,NaN,0,01-01
6,2023-01-07,Saturday,Tennis Shoe,75.0,1,01-01
7,2023-01-08,Sunday,Tennis Shoe,75.0,1,01-08
8,2023-01-09,Monday,0,NaN,0,01-08
9,2023-01-10,Tuesday,0,NaN,0,01-08


In [69]:
year.columns

Index(['Date', 'Day', 'ID', 'Item', 'Color', 'Pattern', 'Category', 'Brand',
       'Bool'],
      dtype='object')

In [94]:
heat_plot = (
    alt.Chart(year, title=f"{item_name} in 2023")
    .mark_rect(
        stroke="white",
        strokeWidth=3,
        opacity=0.85,
    )
    .encode(
        alt.X(
            "Week",
            axis=alt.Axis(
                labelAngle=-60,
#                 sort="month"
            ),
        ),
        alt.Y("Day", sort=weekdays, title=""),
        alt.Color(
            "Bool",
            scale=alt.Scale(domain=[0, 1], range=["#e0ddd5", "#74a675"]),
            legend=None,
        ),
        alt.Tooltip(["Date", "Day"]),
    )
    .properties(height=200, width=600)
    .configure_axis(
        grid=False, domain=False, labelColor="#706f6c", titleColor="#706f6c"
    )
    .configure_title(color="#706f6c").configure_view(strokeWidth=0)
)
heat_plot
#     return heat_plot

alt.Chart(...)

In [20]:
top_10, df = sww.top_10_df()
plot = plot_heatmap(top_10, df,i=5)
plot

alt.Chart(...)

## Scatter 
- wears per season

In [6]:
closet = sww.closet_df()
worn_df = sww.worn(closet)

# read in Google sheet idea
df = pd.read_csv("../data/2023Data.csv").drop("Timestamp", axis=1).melt("Date").dropna()
df["Date"] = pd.to_datetime(df["Date"])
df["ID"] = df.value.str.extract('(\d+)').astype(int)

In [7]:
worn_df

,ID,Name,Count,Item,Category,Sub-Category,Color,Pattern,Brand,Bought,Cost,2023,Price
0,0,Zara Turtleneck,4,Turtleneck,Top,Sweater,Black,Plain,Zara,"Secondhand, Thrifted",cheap,No,0.0
1,1,Plisse Tropical Tank,0,Tropical Tank,Top,Tanktop,"Black, Red, Gold",Feather,Plisse,"Secondhand, Thrifted",cheap,No,0.0
2,2,Plisse Leopard Tank,2,Leopard Tank,Top,Tanktop,"Black, Tan",Leopard,Plisse,"Secondhand, Thrifted",cheap,No,15.0
3,3,Aerie Jeans,11,Jeans,Bottom,Pants,Blue,Plain,Aerie,New,cheap,No,50.0
4,4,Free People Shirt,1,Shirt,Top,Shirt,"Black, White",Cheetah,Free People,"Secondhand, Depop",cheap,No,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,Taylor Swift Baseball Cap,2,Baseball Cap,Accessory,Hat,White,Logo,Taylor Swift,New,pricy,No,35.0
93,93,Davis Farmer'S Market Baseball Cap,0,Baseball Cap,Accessory,Hat,White,Logo,Davis Farmer'S Market,New,cheap,No,0.0
94,94,Ray-Ban Sunglasses,2,Sunglasses,Accessory,Glasses,"Black, Gold",Plain,Ray-Ban,New,expensive,No,160.0
95,95,H&M Tights,1,Tights,Accessory,Tight,Black,Plain,H&M,New,cheap,No,0.0


In [27]:
complete_df = pd.merge(worn_df, df, how="inner", on="ID")
complete_df = complete_df[["ID", "Name", "Count", "Category", "Sub-Category", "Pattern", "Cost", "2023", "Date", "Price"]]
complete_df["CPW"] = (complete_df["Price"] / complete_df["Count"]).round(1)
complete_df["Cost Per Wear"] = "$" + complete_df["CPW"].astype(str) + "0"
complete_df.sample(10)

,ID,Name,Count,Category,Sub-Category,Pattern,Cost,2023,Date,Price,CPW,Cost Per Wear
10,3,Aerie Jeans,11,Bottom,Pants,Plain,cheap,No,2022-12-28,50.0,4.5,$4.50
52,15,Aerie Sports Bra - Navy,1,Top,Workout,Plain,Cheap,No,2023-01-02,25.0,25.0,$25.00
47,11,Zara Coat,4,Outerwear,Coat,"Fur, Houndstooth",Cheap,No,2022-12-27,25.0,6.2,$6.20
66,26,Aerie Leggings - Black,2,Bottom,Workout,Plain,cheap,No,2022-12-21,35.0,17.5,$17.50
112,83,Tj Maxx Square Hoops,1,Accessory,Jewelry,Plain,cheap,No,2022-12-23,0.0,0.0,$0.00
89,73,Lovcat Paris Shoulder Bag,3,Accessory,Bag,Heart,cheap,No,2022-12-29,0.0,0.0,$0.00
104,81,No Brand Gold Hoops,8,Accessory,Jewelry,Plain,pricy,No,2022-12-29,0.0,0.0,$0.00
131,85,Ubc Tote Bag,10,Accessory,Bag,Logo,cheap,No,2022-12-22,20.0,2.0,$2.00
5,2,Plisse Leopard Tank,2,Top,Tanktop,Leopard,cheap,No,2022-12-29,15.0,7.5,$7.50
65,25,Aerie Leggings - Red,1,Bottom,Workout,Plain,cheap,No,2022-12-26,35.0,35.0,$35.00


In [24]:
complete_df["z"] = "$" + complete_df["CPW"].astype(str)
complete_df

,ID,Name,Count,Category,Sub-Category,Pattern,Cost,2023,Date,Price,CPW,Cost Per Wear,z
0,0,Zara Turtleneck,4,Top,Sweater,Plain,cheap,No,2022-12-28,0.0,0.0,$0.0,$0.0
1,0,Zara Turtleneck,4,Top,Sweater,Plain,cheap,No,2022-12-29,0.0,0.0,$0.0,$0.0
2,0,Zara Turtleneck,4,Top,Sweater,Plain,cheap,No,2022-12-31,0.0,0.0,$0.0,$0.0
3,0,Zara Turtleneck,4,Top,Sweater,Plain,cheap,No,2023-01-06,0.0,0.0,$0.0,$0.0
4,2,Plisse Leopard Tank,2,Top,Tanktop,Leopard,cheap,No,2022-12-22,15.0,7.5,$7.5,$7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,92,Taylor Swift Baseball Cap,2,Accessory,Hat,Logo,pricy,No,2023-01-07,35.0,17.5,$17.5,$17.5
150,94,Ray-Ban Sunglasses,2,Accessory,Glasses,Plain,expensive,No,2023-01-07,160.0,80.0,$80.0,$80.0
151,94,Ray-Ban Sunglasses,2,Accessory,Glasses,Plain,expensive,No,2023-01-08,160.0,80.0,$80.0,$80.0
152,95,H&M Tights,1,Accessory,Tight,Plain,cheap,No,2023-01-07,0.0,0.0,$0.0,$0.0


In [45]:
plot_cpw = alt.Chart(complete_df,
              title="2023 Cost Per Wear (CPW)"
             ).mark_circle(
                opacity=0.80,
             ).encode(
                alt.X(
                    "Price",
                    scale=alt.Scale(domain=(0, 200))
                ),
                alt.Y(
                    "Count",
                    scale=alt.Scale(domain=(0, 16)), 
                    title="Times Worn"
                ),
                alt.Color(
                    "Category",
                    scale=alt.Scale(
                        range=["#bb8c9d",
                               "#9a8ca6",
                               "#8ba88a",
                               "#5bccc1",
                               "#e0ddd5"])
                ),
                alt.Size(
                    "CPW",
                    legend=None,
                    scale=alt.Scale(
                        domain=[0, 35]
                    )
                ),
                alt.Tooltip(
                    ["Name", "Category", "Cost Per Wear"]
                )
            ).configure_axis(grid=False)
plot_cpw

alt.Chart(...)

make into function
## DONT USE THIS - updated fx below

In [4]:
def plot_cpw():
    """
    Function for 2023 cost-per-wear plot.
    
    Parameters:
    -----------
        None
         
    Returns:
    --------
        plot : altair.Chart
            Scatter plot of item counts over price.
    """
    
    # read in Google sheet idea
    df = pd.read_csv("../data/2023TestData.csv").drop("Timestamp", axis=1).melt("Date").dropna()
    df["Date"] = pd.to_datetime(df["Date"])
    df["ID"] = df.value.str.extract('(\d+)').astype(int)
    
    # calculate 2023 cost-per-wear
    complete_df = pd.merge(worn_df, df, how="inner", on="ID")
    complete_df = complete_df[["ID", "Name", "Count", "Category", "Sub-Category", "Pattern", "Cost", "2023", "Date", "Price"]]
    complete_df["CPW"] = (complete_df["Price"] / complete_df["Count"]).round(2)
    
    plot = alt.Chart(complete_df,
              title="2023 Cost Per Wear (CPW)"
             ).mark_circle(
                opacity=0.80
             ).encode(
                alt.X(
                    "Price"
                ),
                alt.Y(
                    "Count",
                    title="Times Worn"
                ),
                alt.Color(
                    "Category",
                    scale=alt.Scale(
                        range=["#bb8c9d",
                               "#9a8ca6",
                               "#8ba88a",
                               "#5bccc1",
                               "#e0ddd5"])
                ),
                alt.Size(
                    "CPW",
                    legend=None
                ),
                alt.Tooltip(
                    ["Name", "Category", "CPW"]
                )
            ).configure_axis(grid=False)

    return plot
    

In [2]:
def plot_cpw(worn_df):
    """
    Function for 2023 cost-per-wear plot.

    Parameters:
    -----------
        None

    Returns:
    --------
        plot : altair.Chart
            Scatter plot of item counts over price.
    """

    # read in Google sheet idea
    df = (
        pd.read_csv("../data/2023Data.csv")
        .drop("Timestamp", axis=1)
        .melt("Date")
        .dropna()
    )
    df["Date"] = pd.to_datetime(df["Date"])
    df["ID"] = df.value.str.extract("(\d+)").astype(int)

    # calculate 2023 cost-per-wear
    complete_df = pd.merge(worn_df, df, how="inner", on="ID")
    complete_df = complete_df[
        [
            "ID",
            "Name",
            "Count",
            "Category",
            "Sub-Category",
            "Pattern",
            "Cost",
            "2023",
            "Date",
            "Price",
        ]
    ]
    complete_df["CPW"] = (complete_df["Price"] / complete_df["Count"]).round(2)

    plot = (
        alt.Chart(complete_df, title="2023 Cost Per Wear (CPW)")
        .mark_circle(opacity=0.85)
        .encode(
            alt.X("Price"),
            alt.Y("Count", title="Times Worn"),
            alt.Color(
                "Category",
                scale=alt.Scale(
                    range=[
                        "#bb8c9d",
                        "#9a8ca6",
                        "#8ba88a",
                        "#5bccc1",
                        "#e0ddd5",
                        "#7c9e7b",
                    ]
                ),
            ),
            alt.Size("CPW", legend=None),
            alt.Tooltip(["Name", "Category", "CPW"]),
        )
        .configure_axis(grid=False, labelColor="#706f6c", titleColor="#706f6c")
        .configure_title(color="#706f6c")
    )

    return plot


In [3]:
closet = sww.closet_df()
worn_df = sww.complete_df(closet)
plot_cpw(worn_df)

alt.Chart(...)

## Finalized plots

In [6]:
# read in Google Form data
def plot_mostworn(worn_df):
    most_worn = worn_df.nlargest(15, columns="Count")
    closet_comp = alt.Chart(most_worn, title="2023 Most Worn Pieces"
                       ).mark_bar(
                            color="#6ce4d8",
                            cornerRadiusBottomRight=10,
                            cornerRadiusTopRight=10
                         ).encode(alt.Y("Name",
                                        title="",
                                        axis=alt.Axis(labelAngle=-0), 
                                        sort="-x"),
                                  alt.X("Count", 
                                        title="Times Worn",
                                        axis=alt.Axis(tickMinStep=1)),
                                  alt.Tooltip("Count")
                                 ).configure_axis(grid=False, domain=False).properties(
                                height=300,
                                width=150
                                )
                
    return closet_comp

def plot_facet(worn_df):
    
    categories = ["Top", "Bottom", "Full Body", "Outerwear", "Accessory", "Shoes"]

    cat_plots = []

    for i in categories:
        category_worn = worn_df.loc[worn_df["Category"] == i].nlargest(5, columns="Count")

        category_plot = alt.Chart(category_worn, title=f"2023 Most Worn {i}"
                           ).mark_bar(color="#827191",
                           ).encode(alt.X("Name", title="", axis=alt.Axis(labelAngle=-45), sort="-y"),
                                    alt.Y("Count", 
                                    title="# of Times Worn",
                                    axis=alt.Axis(tickMinStep=1)),
                                    alt.Tooltip(["Name", "Count"])
                            ).properties(
                                height=300,
                                width=150
                                )
        cat_plots.append(category_plot)

    # configure altair charts
    row1 = alt.hconcat(cat_plots[0], cat_plots[1], cat_plots[2])
    row2 = alt.hconcat(cat_plots[3], cat_plots[4], cat_plots[5])

    category_plot = alt.vconcat(row1, row2).configure_axis(grid=False, domain=False)
    return category_plot

def top_10_df(path = "../data/2023Data.csv"):
    """
    Function to return IDs and counts of top 10 most worn items.
    
    Parameters:
    -----------
         path : str
            Path to CSV file containing closet information.
    
    Returns:
    --------
        top_10 : list
            List containing the IDs of the top 10 most worn items.
        
        df : pandas.DataFrame
            Dataframe containing data only for top 10 most worn items.
    """
    df = pd.read_csv(path).drop("Timestamp", axis=1).melt("Date").dropna()
    df["Date"] = pd.to_datetime(df["Date"])
    df["ID"] = df.value.str.extract('(\d+)').astype(int)

    # data wrangling to select top 10 most worn items
    closet = sww.closet_df()
    worn_df = sww.complete_df(closet)
    most_worn = worn_df.nlargest(10, columns="Count")
    
    # merge dataframes
    df = pd.merge(closet, df, how="right", on="ID")
    df = df[["ID", "Item", "Color", "Pattern", "Category", "Date", "Brand"]]

    top_10 = most_worn["ID"].to_list() 
 
    return top_10, df


def plot_heatmap(top_10, df, i=0):
    """
    Function for heatmap plot.
    
    Parameters:
    ----------- 
         top_10 : list
            List containing the IDs of the top 10 most worn items.
            
         df : pandas.DataFrame  
         
    Returns:
    --------
        heatplot : altair.Chart
            Heatmap plot for a single item over a single calender year. 
             
    """
    
    # column of day of week for one calender year 
    time_df = pd.DataFrame()
    time_df["Date"] = pd.date_range(df["Date"].min(), periods=365) 
    time_df["Day"] = time_df["Date"].dt.day_name()
    
    
    heatmap_data = df.loc[df["ID"] == top_10[i]] # need to make this dynamic in plot
    item_name = heatmap_data["Brand"].iloc[0] + " " + heatmap_data["Item"].iloc[0]
    
    # isolate item data
    year = pd.merge(time_df, heatmap_data, how="outer", on="Date")
    year["Item"] = year["Item"].replace(np.nan, 0) 
    year["Bool"] = np.where(year["Item"] == 0, 0, 1)

    # this is horrible to read lol
    # wrangling for prettier plotting 
    week = time_df["Date"].dt.isocalendar()
    year["Week"] = week["week"].fillna(52)
    year["Week"] = year["Week"].fillna(52)
    year['First_day'] = year['Date'] - year['Date'].dt.weekday * np.timedelta64(1, 'D')
    week = time_df["Date"].dt.strftime('%m-%d-%y')
    year["Week"] = year["First_day"].dt.strftime('%m-%d')
    year = year[["Date", "Day", "Item", "ID", "Bool", "Week"]]

    weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]

    heat_plot = alt.Chart(year, title=f"{item_name} in 2023").mark_rect(
    stroke="white",
    strokeWidth=3,
    opacity=0.9,
    ).encode(
    alt.X("Week:O", 
          axis = alt.Axis(labelAngle=-60,                         )
         ), 
    alt.Y("Day", 
          sort=weekdays,
         title=""),
    alt.Color("Bool", 
            scale=alt.Scale(
                domain=[0,1], 
                range=["#e0ddd5", "#74a675"]),
                legend=None
             ),
    alt.Tooltip(["Date", "Day"])
    ).properties(
    height=200,
    width=600
    ).configure_axis(grid=False, domain=False)
    return heat_plot

def plot_cpw():
    """
    Function for 2023 cost-per-wear plot.
    
    Parameters:
    -----------
        None
         
    Returns:
    --------
        plot : altair.Chart
            Scatter plot of item counts over price.
    """
    
    # read in Google sheet idea
    df = pd.read_csv("../data/2023Data.csv").drop("Timestamp", axis=1).melt("Date").dropna()
    df["Date"] = pd.to_datetime(df["Date"])
    df["ID"] = df.value.str.extract('(\d+)').astype(int)
    
    # calculate 2023 cost-per-wear
    complete_df = pd.merge(worn_df, df, how="inner", on="ID")
    complete_df = complete_df[["ID", "Name", "Count", "Category", "Sub-Category", "Pattern", "Cost", "2023", "Date", "Price"]]
    complete_df["CPW"] = (complete_df["Price"] / complete_df["Count"]).round(2)
    
    plot = alt.Chart(complete_df,
              title="2023 Cost Per Wear (CPW)"
             ).mark_circle(
                opacity=0.80
             ).encode(
                alt.X(
                    "Price"
                ),
                alt.Y(
                    "Count",
                    title="Times Worn"
                ),
                alt.Color(
                    "Category",
                    scale=alt.Scale(
                        range=["#bb8c9d",
                               "#9a8ca6",
                               "#8ba88a",
                               "#5bccc1",
                               "#e0ddd5",
                               "#7c9e7b"])
                ),
                alt.Size(
                    "CPW",
                    legend=None
                ),
                alt.Tooltip(
                    ["Name", "Category", "CPW"]
                )
            ).configure_axis(grid=False)

    return plot

In [7]:
closet = sww.closet_df()
worn_df = sww.worn(closet)

In [8]:
plot_mostworn(worn_df)

alt.Chart(...)

In [9]:
sww.plot_facet(worn_df)

alt.VConcatChart(...)

In [10]:
top_10, df = sww.top_10_df()
sww.plot_heatmap(top_10, df, i=2)

alt.Chart(...)

In [10]:
sww.plot_cpw(worn_df)

alt.Chart(...)

I saw an easier way to calculate count on Twitter
- value_count()


# Note to self 

IDEA: 
- is there anyway to read google sheet data in automatically? its a pain to download everytime for testing
- color scheme of plots 
    - `range=["#e0ddd5", "#7c9e7b"]` green/warm gray heatmap
                    range=["#bb8c9d","#9a8ca6","#8ba88a","#5bccc1","#e0ddd5", "#bfae8f"]
